In [7]:
%matplotlib inline
from utils import *

import tensorflow
from tensorflow import keras
from keras import*
# import math
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\mixil\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\mixil\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 2453, in <module>
    from tensorflow.python.util import deprecation
  File "C:\Users\mixil\anaconda3\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 25, in <module>
    from tensorflow.python.platform import tf_logging as logging
ImportError: cannot import name 'tf_logging' from 'tensorflow.python.platform' (C:\Users\mixil\anaconda3\lib\site-packages\tensorflow\python\platform\__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mixil\anaconda3\lib\site-packages\keras\__init__.py", line 3, in <module>
    from tensorflow.keras.layers.experimental.preprocessing imp

TypeError: can only concatenate str (not "list") to str

# Data

#### Load Data

In [4]:
dataset_name = 'dataset_01' 
target = 'distance_1' 
features = 'rssi' 
win_size = None
step_size_ratio = 1

# Tarining
file_names_range = np.arange(20)
train_data = get_data( dataset_name, file_names_range, target, features, win_size=win_size, step_size_ratio=step_size_ratio )

# Test
file_names_range = np.arange(20,25)
test_data = get_data( dataset_name, file_names_range, target, features, win_size=win_size, step_size_ratio=step_size_ratio )

display(data)

,rssi_0,rssi_1,target
0,111.818182,143.727273,0.260033
1,111.818182,143.727273,0.260033
2,111.818182,143.727273,0.260033
3,111.818182,143.727273,0.260033
4,111.818182,143.727273,0.260033
...,...,...,...
7719,81.090909,126.181818,0.270657
7720,81.636364,128.818182,0.269414
7721,82.181818,131.454545,0.268346
7722,82.727273,134.090909,0.267059


In [ ]:
np.random.seed(7)
# load the dataset
dataset = dataframe.values
dataset = dataset.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)



# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))


# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [3]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

NameError: name 'Sequential' is not defined